<a href="https://colab.research.google.com/github/eldananyss21/assignment-3-classification/blob/main/code/assignment3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: Text Classification

In [1]:
# CLONE GITHUB REPO
!git clone https://github.com/eldananyss21/assignment-3-classification.git
%cd assignment-3-classification

Cloning into 'assignment-3-classification'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 55 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 3.17 MiB | 3.34 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/assignment-3-classification


In [2]:
!pip install datasets
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from sklearn import svm
from datasets import Dataset
from torch.optim import Adam
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, XLNetTokenizer, XLNetForSequenceClassification

print("Using GPU:", torch.cuda.is_available())

# IMPORT DATA
train = pd.read_csv("data/train.csv")
dev = pd.read_csv("data/dev.csv")
test = pd.read_csv("data/test.csv")

# ENCODE LABELS
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
dev['label'] = label_encoder.transform(dev['label'])
label_names = label_encoder.classes_.tolist()
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for i, name in enumerate(label_names)}

# COMPUTE METRICS FOR TRAINING PRETRAINED MODELS IN TRACK 2 & 3
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"macro_f1": f1_score(labels, preds, average="macro")}

# TOKENIZE + CREATE DATASETS FOR TRACK 2 & TRACK 3
def tokenize_dataframes(model_name, train, dev, test, train_dev):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_fn(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    train_ds = Dataset.from_pandas(train).map(tokenize_fn, batched=True).rename_column("label", "labels")
    dev_ds = Dataset.from_pandas(dev).map(tokenize_fn, batched=True).rename_column("label", "labels")
    test_ds = Dataset.from_pandas(test).map(tokenize_fn, batched=True)
    train_dev_ds = Dataset.from_pandas(train_dev).map(tokenize_fn, batched=True).rename_column("label", "labels")

    return tokenizer, train_ds, dev_ds, test_ds, train_dev_ds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

## Track 1: Machine Learning Technique

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# 1. TRAIN ON TRAIN SET → EVALUATE ON DEV SET
X_train = model.encode(train['text'].tolist(), show_progress_bar=True)
y_train = label_encoder.transform(train['label'])
X_dev = model.encode(dev['text'].tolist(), show_progress_bar=True)
y_dev = label_encoder.transform(dev['label'])

clf = svm.SVC()
clf.fit(X_train, y_train)
dev_preds = clf.predict(X_dev)
print("Track 1 Dev Macro F1:", f1_score(y_dev, dev_preds, average="macro"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1000 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

Track 1 Dev Macro F1: 0.42722599505243386


In [ ]:
# 2. TRAIN ON TRAIN + DEV SETS → PREDICT ON TEST SET
train_dev = pd.concat([train, dev])
X = model.encode(train_dev['text'].tolist(), show_progress_bar=True)
y = label_encoder.transform(train_dev['label'])
X_test = model.encode(test['text'].tolist(), show_progress_bar=True)

clf.fit(X, y)
test_preds = clf.predict(X_test)
test_labels = label_encoder.inverse_transform(test_preds)

Batches:   0%|          | 0/1125 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame({"id": test["id"], "label": test_labels}).to_csv("track_1_test.csv", index=False)

## Track 2: RoBERTa

In [ ]:
model_name = "roberta-base"
tokenizer, train_ds, dev_ds, test_ds, train_dev_ds = tokenize_dataframes(model_name, train, dev, test, pd.concat([train, dev]))

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id)

args = TrainingArguments(output_dir = "code/results_roberta",
                         eval_strategy="epoch",
                         save_strategy="epoch",
                         learning_rate=1e-5,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                         num_train_epochs=3,
                         weight_decay=0.01,
                         load_best_model_at_end=True,
                         metric_for_best_model="macro_f1",
                         fp16=True,
                         logging_steps=50,
                         save_total_limit=1)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset=dev_ds,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-13-894daf6c0676>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Macro F1
1,1.335100,1.327373,0.427500
2,1.234100,1.312950,0.441897
3,1.161100,1.325728,0.449491


{'eval_loss': 1.3257278203964233,
 'eval_macro_f1': 0.4494908936765253,
 'eval_runtime': 13.0163,
 'eval_samples_per_second': 307.306,
 'eval_steps_per_second': 19.207,
 'epoch': 3.0}

In [ ]:
roberta_preds = trainer.predict(test_ds).predictions.argmax(-1)
labels = [id2label[i] for i in roberta_preds]
pd.DataFrame({"id": test["id"], "label": labels}).to_csv("track_2_test.csv", index=False)

## Track 3: XLNet

In [6]:
model_name = "xlnet-base-cased"
tokenizer, train_ds, dev_ds, test_ds, train_dev_ds = tokenize_dataframes(model_name, train, dev, test, pd.concat([train, dev]))

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id)

args = TrainingArguments(output_dir="code/results_xlnet",
                         eval_strategy="epoch",
                         save_strategy="epoch",
                         learning_rate=1e-5,
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=16,
                         num_train_epochs=2,
                         weight_decay=0.01,
                         load_best_model_at_end=True,
                         metric_for_best_model="macro_f1",
                         fp16=True,
                         logging_steps=50,
                         save_total_limit=1)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset=dev_ds,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-6-4a42cc4160bc>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Macro F1
1,1.365300,1.366394,0.422305
2,1.327700,1.345414,0.428233


{'eval_loss': 1.3454142808914185,
 'eval_macro_f1': 0.4282326786180096,
 'eval_runtime': 74.672,
 'eval_samples_per_second': 53.568,
 'eval_steps_per_second': 3.348,
 'epoch': 2.0}

In [8]:
xlnet_preds = trainer.predict(test_ds).predictions.argmax(-1)
xlnet_labels = [id2label[i] for i in xlnet_preds]
pd.DataFrame({"id": test["id"], "label": xlnet_labels}).to_csv("track_3_test.csv", index=False)

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
max_len = 128

class SubredditDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=max_len,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SubredditDataset(train_df['text'].tolist(), train_df['label'].tolist())
dev_dataset = SubredditDataset(dev_df['text'].tolist(), dev_df['label'].tolist())
test_dataset = SubredditDataset(test_df['text'].tolist())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased',
    num_labels=len(label_encoder.classes_)
)
model = model.to(device)

epochs = 3
optimizer = Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# =============================
# 🔹 Training Loop
# =============================
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
# =============================
# 🔹 Evaluation on Dev Set
# =============================
model.eval()
preds, true = [], []
with torch.no_grad():
    for batch in dev_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true.extend(labels.cpu().numpy())

print("Dev Macro F1:", f1_score(true, preds, average='macro'))

# =============================
# 🔹 Final Training on train + dev and Prediction on Test
# =============================
full_df = pd.concat([train_df, dev_df])
full_dataset = SubredditDataset(full_df['text'].tolist(), full_df['label'].tolist())
full_loader = DataLoader(full_dataset, batch_size=16, shuffle=True)

# Re-init model
model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased',
    num_labels=len(label_encoder.classes_)
)
model = model.to(device)
optimizer = Adam(model.parameters(), lr=2e-5)
model.train()

for epoch in range(epochs):
    loop = tqdm(full_loader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_description(f"Full Train Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

In [ ]:
# Prediction on Test Set
model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        test_preds.extend(preds.cpu().numpy())

# Convert and save
predicted_labels = label_encoder.inverse_transform(test_preds)
submission = pd.DataFrame({"id": test_df["id"], "label": predicted_labels})
submission.to_csv("track_3_test.csv", index=False)
print("✅ track_3_test.csv saved")